In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from dateutil.relativedelta import relativedelta

predictingFY = 18
FY_end = dt.strptime('05/31/2017', '%m/%d/%Y')
pd.set_option('display.max_columns', None) 

In [ ]:
data = pd.read_csv('./data/main-report.csv', dtype='str')
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace(':', '')
data.columns = data.columns.str.replace('.', '')
data.columns = data.columns.str.replace('?', '')

In [ ]:
# create map to store contact giving information
con_giving = {}
for contactId in data['Contact_ID'] :
    con_giving[contactId] = {'Largest_Gift':0, 'Num_Gifts_LFY':0, 'Num_Gifts_2FY':0, 'Num_Gifts_3FY':0, 'Num_Gifts_Past_3_FY':0,
                             'Num_Gifts':0, 'FY_Given':set(), 'FY_Given_in_Past_3_Years':set(), 'Balance_Expected_NFY':0, 'All_Giving': 0, 
                             'Giving_Amt_Past_3_FY':0, 'Avg_Gift_Amount_in_Past_3_Years':0}

In [ ]:
# go through each row to summarize contact information
for row in data.itertuples():
    if not pd.isnull(row.Fiscal_Year):
        rowFY = int(row.Fiscal_Year[-2:])
        if rowFY != predictingFY and rowFY != predictingFY + 1 :
            if (pd.isnull(row.Stage) or row.Stage == 'Completed') and row.Record_Type not in ('Split Gift', 'Pledge', 'Intention', 'Pledge Canceled Value'):  
                con_giving[row.Contact_ID]['FY_Given'].add(rowFY)
                if rowFY < predictingFY and rowFY > predictingFY - 4:
                    con_giving[row.Contact_ID]['FY_Given_in_Past_3_Years'].add(rowFY)
                con_giving[row.Contact_ID]['Num_Gifts'] += 1
                gift_amount = float(row.Analytics_Amount)

                if con_giving[row.Contact_ID]['Largest_Gift'] < float(row.Analytics_Amount):
                    con_giving[row.Contact_ID]['Largest_Gift'] = float(row.Analytics_Amount)
                if rowFY == (predictingFY - 1):  
                    con_giving[row.Contact_ID]['Num_Gifts_LFY'] += 1
                    con_giving[row.Contact_ID]['Num_Gifts_Past_3_FY'] += 1
                    con_giving[row.Contact_ID]['Giving_Amt_Past_3_FY'] += gift_amount
                elif rowFY == (predictingFY - 2):  
                    con_giving[row.Contact_ID]['Num_Gifts_2FY'] += 1
                    con_giving[row.Contact_ID]['Num_Gifts_Past_3_FY'] += 1
                    con_giving[row.Contact_ID]['Giving_Amt_Past_3_FY'] += gift_amount
                elif rowFY == (predictingFY - 3):  
                    con_giving[row.Contact_ID]['Num_Gifts_3FY'] += 1
                    con_giving[row.Contact_ID]['Num_Gifts_Past_3_FY'] += 1
                    con_giving[row.Contact_ID]['Giving_Amt_Past_3_FY'] += gift_amount

                con_giving[row.Contact_ID]['All_Giving'] += gift_amount

        elif rowFY == predictingFY: 
            if row.Record_Type in ('Payment', 'Intention'):
                con_giving[row.Contact_ID]['Balance_Expected_NFY'] += gift_amount

In [ ]:
# now that I'm done using giving data, we can get rid of all duplicate contact Id rows and giving info
data = data.drop_duplicates('Contact_ID', keep="first")
data = data.drop(['Analytics_Amount', 'Campaign_Code', 'Proposal_Opportunity_Name', 
                  'Record_Type', 'Stage', 'Type', 'Fiscal_Year', 'Giving_ID'], axis=1)

In [ ]:
data['New_Donor'] = 0
counter = len(data)
for row in data.itertuples():
    rowIndex = row.Index
    con_id = row.Contact_ID
    num_gifts_past_3_years = con_giving[con_id]['Num_Gifts_Past_3_FY']
    data.at[rowIndex, 'Largest_Gift'] = con_giving[con_id]['Largest_Gift']
    data.at[rowIndex, 'FY_Given_in_Past_3_Years'] = len(con_giving[con_id]['FY_Given_in_Past_3_Years'])
    data.at[rowIndex, 'Num_Gifts_LFY'] = con_giving[con_id]['Num_Gifts_LFY']
    data.at[rowIndex, 'Num_Gifts_2FY'] = con_giving[con_id]['Num_Gifts_2FY']
    data.at[rowIndex, 'Num_Gifts_3FY'] = con_giving[con_id]['Num_Gifts_3FY']
    num_gifts = con_giving[con_id]['Num_Gifts']
    data.at[rowIndex, 'Num_Gifts'] = num_gifts
    data.at[rowIndex, 'Balance_Expected_NFY'] = con_giving[con_id]['Balance_Expected_NFY']
    all_giving = float(con_giving[con_id]['All_Giving'])
    if num_gifts != 0:
        data.at[rowIndex, 'Avg_Gift_Amount'] = all_giving/num_gifts
        data.at[rowIndex, 'Avg_Gifts_Per_Year'] = num_gifts/len(con_giving[con_id]['FY_Given'])
        data.at[rowIndex, 'Num_Gifts_Past_3_Years'] = num_gifts_past_3_years
        if num_gifts_past_3_years != 0:
            giving_amt_past_3_years = con_giving[row.Contact_ID]['Giving_Amt_Past_3_FY']
            data.at[rowIndex, 'Giving_Amt_Past_3_FY'] = giving_amt_past_3_years
            data.at[rowIndex, 'Avg_Gift_Amount_in_Past_3_Years'] = giving_amt_past_3_years/num_gifts_past_3_years
    if pd.notnull(row.First_Gift_Date):
        first_gift_date = dt.strptime(str(row.First_Gift_Date), '%m/%d/%Y')
        if first_gift_date < FY_end:
            days_since = (FY_end - first_gift_date).days
            if days_since <= 365:
                data.at[rowIndex, 'New_Donor'] = 1      
                
    counter -= 1
    print(counter)
data = data.drop(['First_Gift_Date'], axis=1)

In [ ]:
# drop row if the donor is inactive (hasn't given in the past 3 years)
#data = data.drop(data[data.Num_Gifts_Past_3_Years == 0].index)
#data = data.drop(data[pd.isnull(data.Num_Gifts_Past_3_Years)].index)
data.fillna(0)

In [ ]:
# Assign value 0-3 based on prospect priority
data['prospect_priority'] = 0
data.loc[data.Prospect_Priority == 'top', 'prospect_priority'] = 3 
data.loc[data.Prospect_Priority == 'mid', 'prospect_priority'] = 2 
data.loc[data.Prospect_Priority == 'low', 'prospect_priority'] = 1 

data = data.drop(['Prospect_Priority'], axis=1)

In [ ]:
# Assign value based on top constituency for account
data['acct_top_const'] = 0
data.loc[data.Top_Priority_Constituency.isin(['Trustee', 'Trustee Emeritus', 'Former Trustee']), 'acct_top_const'] = 11
data.loc[data.Top_Priority_Constituency.isin(['Alumni-Graduate', 'Non-Person Portfolio', 'Plaque']), 'acct_top_const'] = 10
data.loc[data.Top_Priority_Constituency.isin(['Alumni-Nongraduate']), 'acct_top_const'] = 9
data.loc[data.Top_Priority_Constituency.isin(['Current Parent']), 'acct_top_const'] = 8
data.loc[data.Top_Priority_Constituency.isin(['Parent']), 'acct_top_const'] = 7
data.loc[data.Top_Priority_Constituency.isin(['Grandparent']), 'acct_top_const'] = 6
data.loc[data.Top_Priority_Constituency.isin(['Faculty', 'Former Emp Unkown', 'Former Faculty', 'Former Staff', 'Staff', 'Professor']), 'acct_top_const'] = 5
data.loc[data.Top_Priority_Constituency.isin(['Professor Emeritus']), 'acct_top_const'] = 4
data.loc[data.Top_Priority_Constituency.isin(['Friend', 'Student']), 'acct_top_const'] = 3
data.loc[data.Top_Priority_Constituency.isin(['Fifth Year Program']), 'acct_top_const'] = 2
data.loc[data.Top_Priority_Constituency.isin(['Special Mailing List', 'Women\'s Leadership Council']), 'acct_top_const'] = 1

data = data.drop(['Top_Priority_Constituency'], axis=1)

In [ ]:
# Assign value based on top constituency for account
data['con_top_const'] = 0
data.loc[data.Top_Priority_Constituency1.isin(['Trustee', 'Trustee Emeritus', 'Former Trustee']), 'con_top_const'] = 11
data.loc[data.Top_Priority_Constituency1.isin(['Alumni-Graduate', 'Non-Person Portfolio', 'Plaque']), 'con_top_const'] = 10
data.loc[data.Top_Priority_Constituency1.isin(['Alumni-Nongraduate']), 'con_top_const'] = 9
data.loc[data.Top_Priority_Constituency1.isin(['Current Parent']), 'con_top_const'] = 8
data.loc[data.Top_Priority_Constituency1.isin(['Parent']), 'con_top_const'] = 7
data.loc[data.Top_Priority_Constituency1.isin(['Grandparent']), 'con_top_const'] = 6
data.loc[data.Top_Priority_Constituency1.isin(['Faculty', 'Former Emp Unkown', 'Former Faculty', 'Former Staff', 'Staff', 'Professor']), 'con_top_const'] = 5
data.loc[data.Top_Priority_Constituency1.isin(['Professor Emeritus']), 'con_top_const'] = 4
data.loc[data.Top_Priority_Constituency1.isin(['Friend', 'Student']), 'con_top_const'] = 3
data.loc[data.Top_Priority_Constituency1.isin(['Fifth Year Program']), 'con_top_const'] = 2
data.loc[data.Top_Priority_Constituency1.isin(['Special Mailing List', 'Women\'s Leadership Council']), 'con_top_const'] = 1

data = data.drop(['Top_Priority_Constituency1'], axis=1)

In [ ]:
# Assign value based on account stage
data['stage'] = 0
data.loc[data.Stage1.isin(['Parking Lot', 'Blank Stage']), 'stage'] = -1
data.loc[data.Stage1.isin(['Prospecting', 'Auxiliary Portfolio', 'Volunteer Networker']), 'stage'] = 1
data.loc[data.Stage1.isin(['Discovery']), 'stage'] = 2
data.loc[data.Stage1 == 'Cultivation', 'stage'] = 3
data.loc[data.Stage1 == 'Solicitation', 'stage'] = 4
data.loc[data.Stage1 == 'Stewardship', 'stage'] = 5

data = data.drop(['Stage1'], axis=1)

In [ ]:
# Assign value based on marital status
data['married'] = 0
data.loc[data.Marital_Status == 'Married', 'stage'] = 1
data.loc[data.Marital_Status == 'Widowed', 'stage'] = 2
data = data.drop(['Marital_Status'], axis=1)

In [ ]:
# Make home value null if they couldn't find or calculate
data.loc[data.Home_Value == str(0), 'Home_Value'] = np.nan
data.loc[data.Home_Value_Missing == 'Non SFR', 'Home_Value'] = 0
data = data.drop(['Home_Value_Missing'], axis=1)

In [ ]:
# Make capacity low value null if 0
data.loc[data.Capacity_Low_Value == str(0), 'Capacity_Low_Value'] = np.nan

In [ ]:
# Make age null if 0
data.loc[data.Age == str(0), 'Age'] = np.nan

In [ ]:
# calculate means of gender, capacity low value, and age to fill in null values
ageSum = 0
ageCount = 0
genderSum = 0
genderCount = 0
capacitySum = 0
capacityCount = 0
homeSum = 0
homeCount = 0

data['target_output'] = ''

for row in data.itertuples():
    
    # make target output column
    i = row.Index
    amt = float(row.LFY_Giving)
    output_list = [0] * 12
    if amt >= 1000000.:
        output_list[11] = 1
    elif amt >= 500000.:
        output_list[10] = 1
    elif amt >= 250000.:
        output_list[9] = 1
    elif amt >= 100000.:
        output_list[8] = 1
    elif amt >= 50000.:
        output_list[7] = 1
    elif amt >= 25000.:
        output_list[6] = 1
    elif amt >= 10000.:
        output_list[5] = 1
    elif amt >= 5000.:
        output_list[4] = 1
    elif amt >= 2500.:
        output_list[3] = 1
    elif amt >= 1000.:
        output_list[2] = 1
    elif amt >= 200.:
        output_list[1] = 1
    else:
        output_list[0] = 1
        
    data.at[i, 'target_output'] = output_list
    
    # add number of active communication groups
    split = str(row.Active_Communication_Groups).split('\n')
    data.at[row.Index, 'num_active_comm_groups'] = len(split)
    
    if not pd.isnull(row.Age):
        ageSum += int(row.Age)
        ageCount += 1
    if not pd.isnull(row.Gender):
        genderCount += 1
        if row.Gender == 'Female':
            genderSum += 1
    if not pd.isnull(row.Capacity_Low_Value):
        capacitySum += int(row.Capacity_Low_Value)
        capacityCount += 1
    if not pd.isnull(row.Home_Value):
        homeSum += int(row.Home_Value)
        homeCount += 1
        
data = data.drop(['Active_Communication_Groups', 'LFY_Giving'], axis=1) 

# Assign 0 if male, 1 if female, mean if null
data['gender'] = int(genderSum/genderCount)

data.loc[data.Gender == 'Male', 'gender'] = 0 
data.loc[data.Gender == 'Female', 'gender'] = 1 

data = data.drop(['Gender'], axis=1)

# Assign mean age to null ages
data.loc[pd.isnull(data.Age), 'Age'] = int(ageSum/ageCount)

# Assign mean capacity value to null cells
data.loc[pd.isnull(data.Capacity_Low_Value), 'Capacity_Low_Value'] = int(capacitySum/capacityCount)

# Assign mean home value to null cells
data.loc[pd.isnull(data.Home_Value), 'Home_Value'] = int(homeSum/homeCount)

In [ ]:
# if contact has had a contact report 1, else 0
data['had_contact_report'] = np.where(data['First_Contact_Report_Date'].isnull(), 0, 1)
data = data.drop(['First_Contact_Report_Date'], axis=1)

In [ ]:
# if in portfolio, 1, else 0
data['portfolio_1'] = np.where(data['Primary_Portfolio_Employee_Name'].isnull(), 0, 1)
data = data.drop(['Primary_Portfolio_Employee_Name'], axis=1)

data['portfolio_2'] = np.where(data['Secondary_Portfolio_Employee_Name'].isnull(), 0, 1)
data = data.drop(['Secondary_Portfolio_Employee_Name'], axis=1)

data['portfolio_3'] = np.where(data['Tertiary_Portfolio_Employee_Name'].isnull(), 0, 1)
data = data.drop(['Tertiary_Portfolio_Employee_Name'], axis=1)

In [ ]:
# if account is in a giving society, 1, else 0
data['in_giving_society'] = np.where(data['Giving_Societies'].isnull(), 0, 1)
data = data.drop(['Giving_Societies'], axis=1)

In [ ]:
# if contact graduated from westmont, 1, else 0
data['westmont_grad'] = np.where(data['Westmont_Education_Status'] == 'Graduated', 1, 0)
data = data.drop(['Westmont_Education_Status'], axis=1)

In [ ]:
# add event attendance
event_data = pd.read_csv('./data/event-report.csv', dtype='str')
event_data.columns = event_data.columns.str.replace(' ', '_')
event_data.columns = event_data.columns.str.replace(':', '')

In [ ]:
event_attendance = {}
for contactId in data['Contact_ID'] :
    event_attendance[contactId] = 0

In [ ]:
for row in event_data.itertuples():
    con_id = row.Contact_Contact_ID
    if con_id in event_attendance:
        event_attendance[row.Contact_Contact_ID] += 1

In [ ]:
# add interest/activity count
ia_data = pd.read_csv('./data/IA-report.csv', dtype='str')
ia_data.columns = ia_data.columns.str.replace(' ', '_')
ia_data.columns = ia_data.columns.str.replace(':', '')

In [ ]:
ia = {}
for contactId in data['Contact_ID'] :
    ia[contactId] = {'total':0, 'current':0}

In [ ]:
for row in ia_data.itertuples():
    con_id = row.Contact_Contact_ID
    if con_id in ia:
        if(pd.isnull(row.End_Date)):
            ia[row.Contact_Contact_ID]['total'] += 1
            ia[row.Contact_Contact_ID]['current'] += 1

        else:
            IA_end = dt.strptime(str(row.End_Date), '%m/%d/%Y')
            ia[row.Contact_Contact_ID]['total'] += 1
            if(IA_end < FY_end):
                ia[row.Contact_Contact_ID]['current'] += 1

In [ ]:
# add board/council count
bc_data = pd.read_csv('./data/BC-report.csv', dtype='str')
bc_data.columns = bc_data.columns.str.replace(' ', '_')
bc_data.columns = bc_data.columns.str.replace(':', '')

In [ ]:
bc = {}
for contactId in data['Contact_ID'] :
    bc[contactId] = {'total':0, 'current':0}

for row in bc_data.itertuples():
    con_id = row.Contact_Contact_ID
    if con_id in bc:
        if(pd.isnull(row.End_Date)):
            bc[row.Contact_Contact_ID]['total'] += 1
            bc[row.Contact_Contact_ID]['current'] += 1

        else:
            BC_end = dt.strptime(str(row.End_Date), '%m/%d/%Y')
            bc[row.Contact_Contact_ID]['total'] += 1
            if(BC_end < FY_end):
                bc[row.Contact_Contact_ID]['current'] += 1

In [ ]:
# add contact report information
cr_data = pd.read_csv('./data/CR-report.csv', dtype='str')
cr_data.columns = cr_data.columns.str.replace(' ', '_')
cr_data.columns = cr_data.columns.str.replace(':', '')
cr_data.columns = cr_data.columns.str.replace('?', '')

cr_data = cr_data.drop(cr_data[cr_data['Contact_Purpose'].isin(['No Solicit Verification', 'Removal Verification', 'Demographic Update', 'Alumni Survey', 'Student Ambsdr Interview'])].index)

In [ ]:
# cr data will focus on the 3 years prior to the predicting year
cr = {}
for acctId in data['Account_ID'] :
    cr[acctId] = {'total':0, 'total_significant':0, 'total_in_past_year':0, 'significant_in_past_year':0,
                 'total_in_person':0}

for row in cr_data.itertuples():
    acctId = row.Account_Case_Safe_ID[0:15]
    if acctId in cr:
        cr_date = dt.strptime(str(row.Contact_Date), '%m/%d/%Y')
        if cr_date <= FY_end and cr_date >= (FY_end - relativedelta(years=3)):
            cr[acctId]['total'] += 1
            if cr_date >= (FY_end - relativedelta(years=1)):
                cr[acctId]['total_in_past_year'] += 1
                if row.Significant == 1:
                    cr[acctId]['significant_in_past_year'] += 1
                    cr[acctId]['total_significant'] += 1
            elif row.Significant == 1:
                    cr[acctId]['total_significant'] += 1
            if row.Contact_Type == 'In Person':
                cr[acctId]['total_in_person'] += 1

In [ ]:
# put contact report, event, IA, and BC data into the dataframe
counter = len(data)
for row in data.itertuples():
    rowIndex = row.Index
    con_id = row.Contact_ID
    acct_id = row.Account_ID
    data.at[rowIndex, 'Num_Events_RSVP'] = event_attendance[con_id]
    data.at[rowIndex, 'Num_Total_IA'] = ia[con_id]['total']
    data.at[rowIndex, 'Num_Current_IA'] = ia[con_id]['current']
    data.at[rowIndex, 'Num_Total_BC'] = bc[con_id]['total']
    data.at[rowIndex, 'Num_Current_BC'] = bc[con_id]['current']
    data.at[rowIndex, 'Total_CR'] = cr[acct_id]['total']
    data.at[rowIndex, 'Total_Sig_CR'] = cr[acct_id]['total_significant']
    data.at[rowIndex, 'Total_CR_in_past_year'] = cr[acct_id]['total_in_past_year']
    data.at[rowIndex, 'Sig_in_past_year'] = cr[acct_id]['significant_in_past_year']
    data.at[rowIndex, 'In_person_CR'] = cr[acct_id]['total_in_person']
    counter -= 1

In [ ]:
data = data.drop(['Account_ID'], axis=1) 
data.to_csv('./data/clean-data.csv', index=False)